# EDA: Polycarbonate Prices

This notebook performs exploratory data analysis on polycarbonate (PC) prices from Asian and European suppliers.

In [ ]:
# Import libraries
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Configure paths
from config.paths import RAW_DATA_DIR  # noqa: E402

## Data Loading

We use the [FEDR](https://fred.stlouisfed.org/series/DEXCHUS) exchange rate data to convert prices to a common currency (USD).

In [ ]:
# Load PC price data
pc_price_dir = RAW_DATA_DIR / "pc_price"

df_pc_asia = pd.read_csv(pc_price_dir / "pc_price_asia.csv", sep=";")
df_pc_eu = pd.read_csv(pc_price_dir / "pc_price_eu.csv")

df_conversion_rates = pd.read_csv(RAW_DATA_DIR / "DEXCHUS.csv")

## Cleaning

### Cleaning the names of the columns

In [ ]:
df_pc_eu.columns = df_pc_eu.columns.str.strip()
df_pc_asia.columns = df_pc_asia.columns.str.strip()

### Cleaning the date columns

In [ ]:
df_conversion_rates = df_conversion_rates.copy()
df_conversion_rates["observation_date"] = pd.to_datetime(
    df_conversion_rates["observation_date"], format="%Y-%m-%d"
)

In [ ]:
df_pc_asia = df_pc_asia.copy()
df_pc_asia["Date"] = pd.to_datetime(df_pc_asia["Date"], format="%Y-%m-%d")
df_pc_asia = df_pc_asia.drop(columns=["Year", "Month", "date"])

In [ ]:
df_pc_eu = df_pc_eu.copy()
df_pc_eu["Date"] = pd.to_datetime(df_pc_eu["Date"], format="%Y-%m-%d")
df_pc_eu = df_pc_eu.drop(columns=["Year", "date"])

In [ ]:
# Preview data
print("Asia - First few rows:")
display(df_pc_asia.head())

print("\nEurope - First few rows:")
display(df_pc_eu.head())

Let's look at the range spread in both datasets.

In [ ]:
# Visualize date ranges for both datasets
fig = go.Figure()

# Get date ranges
asia_start = df_pc_asia["Date"].min()
asia_end = df_pc_asia["Date"].max()
eu_start = df_pc_eu["Date"].min()
eu_end = df_pc_eu["Date"].max()

# Add horizontal bars for date ranges
fig.add_trace(
    go.Scatter(
        x=[asia_start, asia_end],
        y=["Asia Dataset", "Asia Dataset"],
        mode="lines",
        name="Asia",
        line=dict(color="coral", width=20),
        hovertemplate="<b>Asia</b><br>%{x|%Y-%m-%d}<extra></extra>",
    )
)

fig.add_trace(
    go.Scatter(
        x=[eu_start, eu_end],
        y=["Europe Dataset", "Europe Dataset"],
        mode="lines",
        name="Europe",
        line=dict(color="steelblue", width=20),
        hovertemplate="<b>Europe</b><br>%{x|%Y-%m-%d}<extra></extra>",
    )
)

# Update layout
fig.update_layout(
    title="PC Price Dataset - Date Range Comparison",
    xaxis_title="Date",
    yaxis_title="Dataset",
    height=400,
    template="plotly_white",
    showlegend=True,
    hovermode="closest",
)

fig.show()

Let's take the exchange rates from FEDR to convert all prices to USD/Kg for the dates in the datasets.

In [ ]:
dates_eu = df_pc_eu["Date"]
dates_asia = df_pc_asia["Date"]
all_pc_dates = pd.to_datetime(np.unique(np.concatenate([dates_eu, dates_asia])))

df_conversion_rates = df_conversion_rates[
    df_conversion_rates["observation_date"].isin(all_pc_dates)
]

## Data Quality Assessment

In [ ]:
# Data size
print(f"Asia - Data shape: {df_pc_asia.shape}")
print(f"Europe - Data shape: {df_pc_eu.shape}")

### Missing Values Analysis

Visually, there appears to be a lot of missing values.

In [ ]:
# Null values analysis
asia_missing_values_percentage = round(
    df_pc_asia.isna().sum() / len(df_pc_asia) * 100, 2
)

eu_missing_values_percentage = round(df_pc_eu.isna().sum() / len(df_pc_eu) * 100, 2)

print("Asia - Missing values percentage:")
display(asia_missing_values_percentage.sort_values(ascending=False))

print("\nEurope - Missing values percentage:")
display(eu_missing_values_percentage.sort_values(ascending=False))

Let's drop columns with too many missing values:

In [ ]:
asia_missing_mask = asia_missing_values_percentage == 100.0
eu_missing_mask = eu_missing_values_percentage == 100.0

# Drop columns with 100% missing values
df_pc_asia = df_pc_asia.loc[:, ~asia_missing_mask]
df_pc_eu = df_pc_eu.loc[:, ~eu_missing_mask]

# For EU, we also drop the unamed column
df_pc_eu = df_pc_eu.drop(columns=["Unnamed: 49"])

### Removing unecessary columns

As discussed with the client, we will drop columns ending with "%".

In [ ]:
cols_to_drop_asia = [col for col in df_pc_asia.columns if col.endswith("%")]
cols_to_drop_eu = [col for col in df_pc_eu.columns if col.endswith("%")]

df_pc_asia = df_pc_asia.drop(columns=cols_to_drop_asia)
df_pc_eu = df_pc_eu.drop(columns=cols_to_drop_eu)

### Fill missing values in conversion rates for Asia

In [ ]:
conversion_map = df_conversion_rates.set_index("observation_date")["DEXCHUS"].to_dict()
mask = df_pc_asia["USD/RMB"].isna() | (df_pc_asia["USD/RMB"] == 0)
df_pc_asia.loc[mask, "USD/RMB"] = df_pc_asia.loc[mask, "Date"].map(conversion_map)

# For remaining missing values, we use forward fill method
df_pc_asia = df_pc_asia.sort_values(by="Date").reset_index(drop=True)
df_pc_asia["USD/RMB"] = df_pc_asia["USD/RMB"].fillna(method="ffill")

## Transformation: Grouping by PC type

The PC prices datasets contain multiple types of polycarbonate, and multiple suppliers for each type. For each PC type, we will consider the minimum price across all suppliers as the representative price for that type (as discussed with the client).

### Europe

In [ ]:
cols_eu_crystal = [col for col in df_pc_eu.columns if "crystal" in col.lower()]
cols_eu_white = [col for col in df_pc_eu.columns if "white" in col.lower()]
cols_eu_gf10 = [
    col
    for col in df_pc_eu.columns
    if ("gf10" in col.lower() or "gf 10" in col.lower())
    and "recyclé" not in col.lower()
]
cols_eu_gf20 = [col for col in df_pc_eu.columns if "gf20" in col.lower()]
cols_eu_si = [col for col in df_pc_eu.columns if "si" in col.lower()]
cols_eu_recycled_white = [
    col
    for col in df_pc_eu.columns
    if "recyclé" in col.lower() and "white" in col.lower()
]
cols_eu_recycled_grey = [
    col
    for col in df_pc_eu.columns
    if "recyclé" in col.lower() and "grey" in col.lower()
]

In [ ]:
df_pc_eu["PC_Crystal_best"] = df_pc_eu[cols_eu_crystal].min(axis=1)
df_pc_eu["PC_White_best"] = df_pc_eu[cols_eu_white].min(axis=1)
df_pc_eu["PC_GF10_best"] = df_pc_eu[cols_eu_gf10].min(axis=1)
df_pc_eu["PC_GF20_best"] = df_pc_eu[cols_eu_gf20].min(axis=1)
df_pc_eu["PC_Si_best"] = df_pc_eu[cols_eu_si].min(axis=1)
df_pc_eu["PC_Recycled_white_best"] = df_pc_eu[cols_eu_recycled_white].min(axis=1)
df_pc_eu["PC_Recycled_grey_best"] = df_pc_eu[cols_eu_recycled_grey].min(axis=1)

df_pc_eu = df_pc_eu.drop(
    columns=cols_eu_crystal
    + cols_eu_white
    + cols_eu_gf10
    + cols_eu_gf20
    + cols_eu_si
    + cols_eu_recycled_white
    + cols_eu_recycled_grey
)

In [ ]:
df_pc_eu

### Asia

We want the same currency for all prices, so we will convert all prices to USD/Kg.

In [ ]:
# Get all price columns (exclude Date, USD/RMB, and non-price columns)
price_columns = [
    col
    for col in df_pc_asia.columns
    if col not in ["Date", "USD/RMB", "Year", "Month", "date"]
    and "SPREAD" not in col
    and "ICIS" not in col
    and "CHEMSINO" not in col
    and "TECNON" not in col
]


def get_supplier_pc_info(col_name):
    """Extract supplier name and PC type from column name."""
    # Extract PC type (GP, FR, GF, etc.)
    pc_type = None
    for pt in [
        "GP recycled",
        "GP",
        "GF10FR",
        "GF recycled",
        "GF",
        "FR",
        "NAT",
        "Si recycled",
        "Si",
    ]:
        if pt in col_name:
            pc_type = pt
            break

    # Extract supplier name
    supplier = None
    for i in range(1, 10):
        if f"asia_supplier_{i}" in col_name:
            supplier = f"asia_supplier_{i}"
            break

    return supplier, pc_type


converted_cols = {}
conversion_stats = {
    "kept_usd_kg": 0,
    "converted_rmb_t": 0,
    "converted_rmb_kg": 0,
    "already_usd": 0,
    "errors": [],
}

# Process each column
for col in price_columns:
    supplier, pc_type = get_supplier_pc_info(col)

    # Determine unit from column name
    if "RMB/T" in col or "RMB/t" in col:
        unit = "RMB/T"
    elif "RMB/KG" in col or "RMB/kg" in col or "RMB/Kg" in col:
        unit = "RMB/KG"
    elif "USD/KG" in col or "USD/kg" in col or "USD/Kg" in col:
        unit = "USD/KG"
    elif "INR/KG" in col or "INR/kg" in col or "INR/Kg" in col:
        unit = "INR/KG"
    else:
        # Default assumption: USD/Kg if no unit specified
        unit = "USD/KG"

    # Handle columns without supplier name (like "PC Si (RMB/kg)")
    if supplier is None or pc_type is None:
        # Try to extract just the PC type for columns without supplier
        if " Si recycled " in col or col.startswith("PC Si recycled"):
            pc_type_only = "Si recycled"
        elif " Si " in col or col.startswith("PC Si"):
            pc_type_only = "Si"
        else:
            # Can't parse this column
            if unit == "INR/KG":
                print(f"⚠️  Skipping {col} - INR conversion not implemented")
            else:
                print(f"⚠️  Skipping {col} - couldn't parse supplier/type")
            conversion_stats["errors"].append(col)
            continue

        # Create column name without supplier
        new_col_name = f"PC {pc_type_only} (USD/Kg)"
    else:
        # Create standardized column name with supplier
        new_col_name = f"{supplier} PC {pc_type} (USD/Kg)"

    # Skip INR conversions
    if unit == "INR/KG":
        print(f"⚠️  Skipping {col} - INR conversion not implemented")
        conversion_stats["errors"].append(col)
        continue

    # Convert based on unit
    if unit == "USD/KG":
        # Already in USD/Kg, just copy
        converted_cols[new_col_name] = df_pc_asia[col].copy()
        conversion_stats["already_usd"] += 1
    elif unit == "RMB/T":
        # Convert from RMB/T to USD/Kg
        # RMB/T -> USD/T (divide by USD/RMB) -> USD/Kg (divide by 1000)
        converted_cols[new_col_name] = df_pc_asia[col] / df_pc_asia["USD/RMB"] / 1000
        conversion_stats["converted_rmb_t"] += 1
    elif unit == "RMB/KG":
        # Convert from RMB/Kg to USD/Kg
        converted_cols[new_col_name] = df_pc_asia[col] / df_pc_asia["USD/RMB"]
        conversion_stats["converted_rmb_kg"] += 1

# For duplicate columns (same supplier, same PC type), prefer USD/Kg values when
# available
final_cols = {}
for col_name, values in converted_cols.items():
    if col_name in final_cols:
        # Merge: prefer non-null values from USD/Kg column
        final_cols[col_name] = values.combine_first(final_cols[col_name])
        conversion_stats["kept_usd_kg"] += 1
    else:
        final_cols[col_name] = values

# Create new dataframe starting with original columns
df_pc_asia_converted = df_pc_asia.copy()

# Track which original columns were converted
converted_original_cols = []
for col in price_columns:
    supplier, pc_type = get_supplier_pc_info(col)

    # Try both with and without supplier names
    possible_new_names = []

    if supplier is not None and pc_type is not None:
        possible_new_names.append(f"{supplier} PC {pc_type} (USD/Kg)")

    # Also check for columns without supplier (like "PC Si (USD/Kg)")
    if " Si recycled " in col or col.startswith("PC Si recycled"):
        possible_new_names.append("PC Si recycled (USD/Kg)")
    elif " Si " in col or col.startswith("PC Si"):
        possible_new_names.append("PC Si (USD/Kg)")

    # Check if any of the possible names exists in final_cols
    for new_col_name in possible_new_names:
        if new_col_name in final_cols:
            converted_original_cols.append(col)
            break

# Remove only the columns that were successfully converted
df_pc_asia_converted = df_pc_asia_converted.drop(columns=converted_original_cols)

# Add all converted price columns
for col_name, values in sorted(final_cols.items()):
    df_pc_asia_converted[col_name] = values

# Replace original dataframe
df_pc_asia = df_pc_asia_converted.copy()

print("\n✅ Conversion complete!")
print("\nConversion statistics:")
print(f"  - Already in USD/Kg: {conversion_stats['already_usd']} columns")
print(f"  - Converted from RMB/T: {conversion_stats['converted_rmb_t']} columns")
print(f"  - Converted from RMB/Kg: {conversion_stats['converted_rmb_kg']} columns")
print(f"  - Merged duplicate USD/Kg: {conversion_stats['kept_usd_kg']} columns")
if conversion_stats["errors"]:
    print(f"  - Errors/Skipped: {len(conversion_stats['errors'])} columns (kept as-is)")

print("\n📊 Final dataset:")
print(f"  - Shape: {df_pc_asia.shape}")
print(f"  - Converted columns: {len(final_cols)}")
print(f"  - Original columns removed: {len(converted_original_cols)}")
print(
    f"  - Columns kept unchanged: {df_pc_asia.shape[1] - len(final_cols) - 2}"
)  # -2 for Date and USD/RMB
print(f"  - Date range: {df_pc_asia['Date'].min()} to {df_pc_asia['Date'].max()}")

In [ ]:
# Converted price columns (USD/Kg)
price_cols_asia = [
    col
    for col in df_pc_asia.columns
    if col.endswith("(USD/Kg)") and col not in ["Date", "USD/RMB"]
]

# GP
cols_asia_gp = [
    col for col in price_cols_asia if " GP " in col and "recycled" not in col.lower()
]

# GP Recycled
cols_asia_gp_recycled = [
    col for col in price_cols_asia if " GP " in col and "recycled" in col.lower()
]

# FR
cols_asia_fr = [
    col
    for col in price_cols_asia
    if " FR " in col and "GF" not in col  # Exclude GF10FR
]

# GF - includes GF10FR
cols_asia_gf = [
    col
    for col in price_cols_asia
    if (" GF " in col or "GF10FR" in col) and "recycled" not in col.lower()
]

# GF Recycled
cols_asia_gf_recycled = [
    col for col in price_cols_asia if " GF " in col and "recycled" in col.lower()
]

# NAT
cols_asia_nat = [col for col in price_cols_asia if " NAT " in col]

# Si
cols_asia_si = [
    col for col in price_cols_asia if " Si " in col and "recycled" not in col.lower()
]

# Si Recycled
cols_asia_si_recycled = [
    col for col in price_cols_asia if " Si " in col and "recycled" in col.lower()
]

In [ ]:
df_pc_asia["PC_GP_best"] = df_pc_asia[cols_asia_gp].min(axis=1)
df_pc_asia["PC_GP_Recycled_best"] = df_pc_asia[cols_asia_gp_recycled].min(axis=1)
df_pc_asia["PC_FR_best"] = df_pc_asia[cols_asia_fr].min(axis=1)
df_pc_asia["PC_GF_best"] = df_pc_asia[cols_asia_gf].min(axis=1)
df_pc_asia["PC_GF_Recycled_best"] = df_pc_asia[cols_asia_gf_recycled].min(axis=1)
df_pc_asia["PC_NAT_best"] = df_pc_asia[cols_asia_nat].min(axis=1)
df_pc_asia["PC_Si_best"] = df_pc_asia[cols_asia_si].min(axis=1)
df_pc_asia["PC_Si_Recycled_best"] = df_pc_asia[cols_asia_si_recycled].min(axis=1)

df_pc_asia = df_pc_asia.drop(
    columns=cols_asia_gp
    + cols_asia_gp_recycled
    + cols_asia_fr
    + cols_asia_gf
    + cols_asia_gf_recycled
    + cols_asia_nat
    + cols_asia_si
    + cols_asia_si_recycled
)

## Analysis

In [ ]:
print("Cleaned Asia PC Prices DataFrame:")
display(df_pc_asia.head())

print("\nCleaned Europe PC Prices DataFrame:")
display(df_pc_eu.head())

### Plots of best prices over time per PC type

#### Europe

In [ ]:
fig = go.Figure()

# Get all PC type columns (ending with "_best")
pc_type_cols_eu = [col for col in df_pc_eu.columns if col.endswith("_best")]

# Define color palette
colors = [
    "#1f77b4",  # blue
    "#ff7f0e",  # orange
    "#2ca02c",  # green
    "#d62728",  # red
    "#9467bd",  # purple
    "#8c564b",  # brown
    "#e377c2",  # pink
]

# Add trace for each PC type
for i, col in enumerate(pc_type_cols_eu):
    # Clean up column name for legend
    pc_type_name = col.replace("PC_", "").replace("_best", "").replace("_", " ")

    fig.add_trace(
        go.Scatter(
            x=df_pc_eu["Date"],
            y=df_pc_eu[col],
            mode="lines+markers",
            name=pc_type_name,
            line=dict(width=2, color=colors[i % len(colors)]),
            marker=dict(size=4),
            hovertemplate="<b>%{fullData.name}</b><br>"
            + "Date: %{x|%Y-%m-%d}<br>"
            + "Price: ?%{y:.2f}/kg<br>"
            + "<extra></extra>",
        )
    )

# Update layout
fig.update_layout(
    title="European PC Prices Evolution by Type",
    xaxis_title="Date",
    yaxis_title="Price (?/kg)",
    hovermode="x unified",
    height=600,
    template="plotly_white",
    legend=dict(
        title="PC Type",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.8)",
        bordercolor="lightgray",
        borderwidth=1,
    ),
)

fig.show()

#### Asia

In [ ]:
fig = go.Figure()

# Get all PC type columns (ending with "_best")
pc_type_cols_asia = [col for col in df_pc_asia.columns if col.endswith("_best")]

# Define color palette
colors = [
    "#1f77b4",  # blue
    "#ff7f0e",  # orange
    "#2ca02c",  # green
    "#d62728",  # red
    "#9467bd",  # purple
    "#8c564b",  # brown
    "#e377c2",  # pink
    "#bcbd22",  # olive
]

# Add trace for each PC type
for i, col in enumerate(pc_type_cols_asia):
    # Clean up column name for legend
    pc_type_name = col.replace("PC_", "").replace("_best", "").replace("_", " ")

    fig.add_trace(
        go.Scatter(
            x=df_pc_asia["Date"],
            y=df_pc_asia[col],
            mode="lines+markers",
            name=pc_type_name,
            line=dict(width=2, color=colors[i % len(colors)]),
            marker=dict(size=4),
            hovertemplate="<b>%{fullData.name}</b><br>"
            + "Date: %{x|%Y-%m-%d}<br>"
            + "Price: €%{y:.2f}/kg<br>"
            + "<extra></extra>",
        )
    )

# Update layout
fig.update_layout(
    title="Asian PC Prices Evolution by Type",
    xaxis_title="Date",
    yaxis_title="Price (USD/kg)",
    hovermode="x unified",
    height=600,
    template="plotly_white",
    legend=dict(
        title="PC Type",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.8)",
        bordercolor="lightgray",
        borderwidth=1,
    ),
)

fig.show()

- GF Recycled prices are false
- GP prices always decreasing?
- Si prices higher than other types --> Coherent with EU

### Correlation Analysis Between columns of the datasets

#### Europe

In [ ]:
# Correlation Analysis - Europe Dataset
# Get numeric columns (exclude Date)
numeric_cols_eu = [
    col
    for col in df_pc_eu.columns
    if col != "Date" and df_pc_eu[col].dtype in ["float64", "int64"]
]

# Calculate correlation matrix
corr_matrix_eu = df_pc_eu[numeric_cols_eu].corr()

# Create figure
plt.figure(figsize=(12, 10))

# Create heatmap
sns.heatmap(
    corr_matrix_eu,
    annot=True,
    fmt=".2f",
    cmap="RdBu_r",
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8, "label": "Correlation"},
    vmin=-1,
    vmax=1,
)

plt.title(
    "Correlation Matrix - European PC Prices", fontsize=16, fontweight="bold", pad=20
)
plt.xlabel("")
plt.ylabel("")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

#### Asia

In [ ]:
# Correlation Analysis - Asia Dataset
# Get numeric columns (exclude Date)
numeric_cols_asia = [
    col
    for col in df_pc_asia.columns
    if col != "Date" and df_pc_asia[col].dtype in ["float64", "int64"]
]

# Calculate correlation matrix
corr_matrix_asia = df_pc_asia[numeric_cols_asia].corr()

# Create figure
plt.figure(figsize=(12, 10))

# Create heatmap
sns.heatmap(
    corr_matrix_asia,
    annot=True,
    fmt=".2f",
    cmap="RdBu_r",
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8, "label": "Correlation"},
    vmin=-1,
    vmax=1,
)

plt.title(
    "Correlation Matrix - European PC Prices", fontsize=16, fontweight="bold", pad=20
)
plt.xlabel("")
plt.ylabel("")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### Self correlation analysis

#### Europe

In [ ]:
# Autocorrelation Analysis - Europe Dataset
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Get PC type columns
pc_type_cols_eu = [col for col in df_pc_eu.columns if col.endswith("_best")]

# Create subplots for ACF and PACF for each PC type
n_types = len(pc_type_cols_eu)
fig, axes = plt.subplots(n_types, 2, figsize=(14, 4 * n_types))

if n_types == 1:
    axes = axes.reshape(1, -1)

for i, col in enumerate(pc_type_cols_eu):
    # Get non-null values
    series = df_pc_eu[col].dropna()

    # Clean up column name for title
    pc_type_name = col.replace("PC_", "").replace("_best", "").replace("_", " ")

    # Determine maximum lags based on series length
    # ACF can use more lags, PACF limited to 50% of sample size
    max_acf_lags = min(40, len(series) - 1)
    max_pacf_lags = min(40, len(series) // 2 - 1)

    # Plot ACF
    plot_acf(series, lags=max_acf_lags, ax=axes[i, 0], alpha=0.05)
    axes[i, 0].set_title(f"ACF - {pc_type_name}", fontweight="bold")
    axes[i, 0].set_xlabel("Lag")
    axes[i, 0].set_ylabel("Autocorrelation")

    # Plot PACF
    plot_pacf(series, lags=max_pacf_lags, ax=axes[i, 1], alpha=0.05)
    axes[i, 1].set_title(f"PACF - {pc_type_name}", fontweight="bold")
    axes[i, 1].set_xlabel("Lag")
    axes[i, 1].set_ylabel("Partial Autocorrelation")

    # Add data availability info
    axes[i, 0].text(
        0.02,
        0.98,
        f"n={len(series)} ({len(series) / len(df_pc_eu) * 100:.1f}% available)",
        transform=axes[i, 0].transAxes,
        verticalalignment="top",
        bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5),
        fontsize=8,
    )

plt.tight_layout()
plt.show()

  - **Crystal**: Differenciate once

# Next steps

- Convert transformations into a pipeline 
- Complete data with info from other datasets --> Feature engineering and selection
- Modeling